In [11]:
import openai
from dotenv import load_dotenv
import os
from IPython.display import display, HTML, JSON, Markdown
from langchain import PromptTemplate
from langchain.llms import AzureOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_COMPLETION_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

client = AzureOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version=OPENAI_DEPLOYMENT_VERSION
)

In [12]:
def init_llm(model=OPENAI_MODEL_NAME,
             deployment_name=OPENAI_DEPLOYMENT_NAME,
             openai_api_version=OPENAI_DEPLOYMENT_VERSION,
             azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT,
             temperature=0,
             max_tokens=400,
             top_p=1,
             ):

    llm = AzureOpenAI(deployment_name=deployment_name,
                      model=model,
                      openai_api_version=openai_api_version,
                      temperature=temperature,
                      azure_endpoint=azure_endpoint,
                      max_tokens=max_tokens,
                      top_p=top_p,
                      model_kwargs={"stop": ["<|im_end|>"]}
                      )
    return llm
llm = init_llm()

### **Retain conversation history** 

##### The Large Language Models (LLMs) are <ins>**stateless**</ins>. This means that they don’t retain any information about the conversation history.
Each transaction is independent of the previous one. Chatbots keep in memory the conversation history and use it to generate the next response. This is why they are able to generate more coherent responses.

Previously we saw that a model fails to answer the question that requires context. We can solve this problem by retaining the conversation history. We can do this by using the LangChain ConversationBufferMemory.

In [13]:
template = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
            
            USER: {question}
            ASSISTANT:

            <|im_end|>
            """

In [14]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate

# ConversationBufferMemory is a memory that stores the conversation history
memory = ConversationBufferMemory()
# try to change the verbose to True, to see more details
conversation = ConversationChain(llm=llm, memory=memory, verbose=False)

In [15]:
prompt = ChatPromptTemplate.from_template(template=template)

response = conversation.run(input=prompt.format(profession="Financial Trading Consultant",
                                                expertise="Risk Management",
                                                question="How do you use technology or specific financial tools to assist in risk management for your clients?"))

display(Markdown(response))

 Hello! As a Financial Trading Consultant specializing in Risk Management, I use a variety of technology and financial tools to assist my clients. One of the main tools I use is a risk management software, which helps me analyze and monitor potential risks in my clients' portfolios. This software uses advanced algorithms and data analysis to identify potential risks and provide recommendations for mitigating them. Additionally, I also use financial modeling tools to simulate different scenarios and assess the impact of potential risks on my clients' investments. These tools allow me to make informed decisions and develop effective risk management strategies for my clients. Is there anything else you would like to know about my use of technology and financial tools in risk management?

In [6]:
# Now with the conversation history, the model can answer questions that require context.
response = conversation.run(input=prompt.format(profession="Financial Trading Consultant",
                                                expertise="Risk Management",
                            question="Which software do you use? List software products in a separate line."))

display(Markdown(response))

 Sure, I primarily use risk management software such as Riskalyze, RiskMetrics, and RiskVision. These are all widely used and trusted tools in the financial industry for risk management. I also use financial modeling software such as Monte Carlo simulation tools and Value at Risk (VaR) calculators to assess and manage risks for my clients. Is there anything else you would like to know about the specific software I use?

In [7]:
# you can print the conversation history buffer:
history = conversation.memory.chat_memory.messages
for msg in history:
    print(f"{msg.type}: {msg.content}")

human: Human: You are a Financial Trading Consultant answering users questions. 
            More specifically, you are an expert in Risk Management. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.
            
            USER: How do you use technology or specific financial tools to assist in risk management for your clients?
            ASSISTANT:

            <|im_end|>
            
ai:  Hello! As a Financial Trading Consultant specializing in Risk Management, I use a variety of technology and financial tools to assist my clients. One of the main tools I use is a risk management software, which helps me analyze and monitor potential risks in my clients' portfolios. This software uses advanced algorithms and data analysis to identify potential risks and provide recommendations for mitigating them. Addition

In [22]:
# pay attention that LangChain added to the prompt: ```Current conversation:{history}```
print(conversation.prompt)
print()
print(memory.load_memory_variables({}))

input_variables=['history', 'input'] template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

{'history': "Human: Human: You are a Financial Trading Consultant answering users questions. \n            More specifically, you are an expert in Risk Management. Answer in a clear and concise manner. Assume that the user is not a subject expert.\n            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.\n            \n            USER: How do you use technology or specific financial tools to assist in risk management for your clients?\n            ASSISTANT:\n\n            <|im_end|>\n            \nAI:  Hello! As a Financial Trading Consultant specializing in Risk Mana

In [27]:
ans = conversation.run(input=prompt.format(profession="Financial Trading Consultant",
                                           expertise="Risk Management",
                                           question="List all questions I've asked you about Risk Management?"))
display(Markdown(ans))

   Sure, I can list all the questions you have asked me about Risk Management. So far, you have asked me about how I use technology and financial tools to assist in risk management for my clients. Is there anything else you would like to know?

#### More conversation memory types.

Keeping full conversation history could be expensive and we can hit the Azure API limits. LangChain provides different types of conversation memory that can be used to keep the conversation history and mitigate the limits issues.

**1. ConversationBufferWindowMemory** - keeps the last N messages in the conversation history.

**2. ConversationTokenBufferMemory** - keeps the last N tokens in the conversation history.

**3. ConversationSummaryBufferMemory** - keeps summary of the conversation over time.

Additional Memory Type includes:

**4. Vector data memory** - stores the text in a vector DB and retrieves most semantically similar text.

**5. Entity memories** - use LLM which rememebers details about specific entities. 

Note: You can use multiple memories at the same time. 

Finally you can store the conevrsation history in a conventional database like SQL or NoSQL. 


In [29]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "hi"}, {"output": "hello"})
memory.save_context({"input": "What's your name"},
                    {"output": "My name is John"})

memory.load_memory_variables({})

{'history': "Human: What's your name\nAI: My name is John"}